In [9]:
import json
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForQuestionAnswering, T5ForQuestionAnswering, AutoTokenizer, T5Tokenizer
#from transformers import T5ForConditionalGeneration, AutoTokenizer
from GPUtil import showUtilization as gpu_usage
from numba import cuda


In [6]:
class genshin_impact(Dataset):
    def __init__(self, file_path):
        self.data = self.load_data(file_path)
        self.tokenizer = AutoTokenizer.from_pretrained(qa_model)

    def load_data(self, file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)
        paragraphs = data['data'][0]['paragraphs']
        extracted_data = []
        for paragraph in paragraphs:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text']
                start_pos = qa['answers'][0]['answer_start']
                end_pos = qa['answers'][0]['answer_end']
                extracted_data.append({
                    'context': context,
                    'question': question,
                    'answer': answer,
                    'start_pos': start_pos,
                    'end_pos': end_pos
                })
        return extracted_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        example = self.data[index]
        question = example['question']
        context = example['context']
        answer = example['answer']
        inputs = self.tokenizer.encode_plus(question, context, add_special_tokens=True, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        start_pos = torch.tensor(example['start_pos'])
        end_pos = torch.tensor(example['end_pos'])
        return input_ids, attention_mask, start_pos, end_pos

In [7]:
file_path = './gi_squad_ds/gi_answers.json'
dataset = genshin_impact(file_path)

In [8]:
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  3% |


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = T5ForQuestionAnswering.from_pretrained(qa_model)
#model = T5ForQuestionAnswering.from_pretrained(ckpt).to(device)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()
batch_size = 4
num_epochs = 100

# Create data loader
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at mrm8488/spanish-t5-small-sqac-for-qa and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in data_loader:
        # Move batch tensors to the device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        start_positions = batch[2].to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss:.4f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 3.81 GiB total capacity; 3.09 GiB already allocated; 15.12 MiB free; 3.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF